# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [17]:
from os import walk, getcwd
from ticktack.fitting import SingleFitter, MultiFitter
from ticktack import load_presaved_model
from math import exp

In [3]:
models = { # This dictionary contains the units for the fluxes and production function
    "Guttler14": {  # Units of the Guttler 2014 paper
        "production_rate_units": "atoms/cm^2/s",    # Units of the production rate 
        "flow_rate_units": "Gt/yr"                  # Units of the fluxes
    }
}

In [6]:
data_sets = []  # List for storing the data set locations
data_sets_directory = f"{getcwd()}/datasets/775AD" # Home directory of the data 
for (root, dirs, files) in walk(data_sets_directory):    # Looping over directories 
    for file in files:  # Looping through the files 
        file_path = root + "/" + file   # Setting up the path 
        data_sets.append(file_path)  # Extending the stored directoriess

In [ ]:
def production_function(time, *parameters):
    """
    Given the time and parameters of the production function. Generates the total production of carbon 14 at that time using a linear model and a super-gaussian pulse. 
    Parameters:
        time: float -> The time in years.
        parameters: tuple -> The parameters of the production function. In this case, steady state, long term trend, middle, duration and amplitude.
    Returns:
        float -> The carbon 14 production at time.
    """
    const, lin_coeff, center, duration, amplitude = parameters
    super_gaussian = amplitude * exp(((time - center) / (duration / 2)) ** 8)
    long_term_trend = const + lin_coeff * time
    return super_gaussian + long_term_trend

In [ ]:
def get_model(model: str, datum: str):
    """
    Using a `ticktack.SingleFitter` this function compiles the model and selects the appropriate production function.
    """
    cbm = load_presaved_model( # Generating the CarbobBoxModel using ticktack
        model,  # Name of the model as looped from the models dictionary 
        production_rate_units=models[model]["production_rate_units"], 
        flow_rate_units=models[model]["flow_rate_units"]
    )

    bayesian_model = SingleFitter(cbm)   # Fitting a model 
    bayesian_model.prepare_function(production_function)    # Generating the simple sin model
    bayesian_model.load_data(datum)   # Loading the data into the model  

    return bayesian_model